# Project Notebook

### Imports

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Add, Input
from tensorflow.keras.models import Model
from tensorflow.nn import depth_to_space
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint

2022-04-26 17:31:32.655636: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 17:31:32.655659: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from utils import downsample, normalize
from SRCNN import srcnn
import DIPCI

### Path & Data

In [3]:
PATH_DATA = "./Data"

Load Data

In [4]:
data_ssh = PATH_DATA+"/1308_square_NATL60_SSH_R09.npy"
data_sst = PATH_DATA+"/1308_square_NATL60_SST_R09.npy"
ssh = np.load(data_ssh)
sst = np.load(data_sst)

HR Data

In [5]:
ssh_hr_train = ssh[0:366, :, :]
ssh_hr_test  = ssh[366:, :, :]
sst_hr_train = sst[0:366, :, :]
sst_hr_test  = sst[366:, :, :]

LR Data

In [6]:
downsampling_factor = 2
ssh_lr_train = np.array([downsample(img, downsampling_factor) for img in ssh_hr_train])
ssh_lr_test  = np.array([downsample(img, downsampling_factor) for img in ssh_hr_test])

Normalize (SSH)

In [7]:
ssh_hr_train = np.array([normalize(img) for img in ssh_hr_train])
ssh_lr_train = np.array([normalize(img) for img in ssh_lr_train])
ssh_hr_test  = np.array([normalize(img) for img in ssh_hr_test])
ssh_lr_test  = np.array([normalize(img) for img in ssh_lr_test])

### SRCNN 

In [8]:
epochs = 100
loss_fn = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=0.003)
checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
callbacks = [checkpoint]

In [9]:
SRCNN = srcnn()

2022-04-26 17:31:35.201006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-26 17:31:35.201040: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-26 17:31:35.201061: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3800411-Latitude-E7270): /proc/driver/nvidia/version does not exist
2022-04-26 17:31:35.201300: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
SRCNN.compile(optimizer=optimizer, loss=loss_fn)
SRCNN.fit( ssh_lr_train, epochs=epochs, callbacks=callbacks, validation_data=ssh_hr_train.all(), verbose=1 )

Epoch 1/100


ValueError: in user code:

    File "/home/matt/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/matt/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/matt/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/matt/.local/lib/python3.8/site-packages/keras/engine/training.py", line 861, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/matt/.local/lib/python3.8/site-packages/keras/engine/training.py", line 818, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.MeanSquaredError object at 0x7fefb2dd1400>, and therefore expects target data to be

### Model

In [ ]:
filters = 64
upscale_factor = 2

In [ ]:
def simple_conv_model( shape=1, filters=filters ):
    ''' extracts SST features '''
    input_img = Input(shape=(None, None, shape))
    x = Conv2D(filters, (3,3), padding='same')(input_img)
    x = LeakyReLU(alpha=0.2)(x)        
    x = Conv2D(filters, (3,3), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    return Model(inputs=input_img, outputs=x)

In [ ]:
def shuffle_model( filters=filters ):
    ''' upsamples in feature branch '''
    input_img = Input(shape=(None, None, filters))
    x = Conv2D(filters, (3,3), padding='same')(input_img)
    x = depth_to_space(x, upscale_factor)        
    x = Conv2D(filters, (2,2), padding='same')(x)
    x = LeakyReLU()(x)
    return Model(inputs=input_img, outputs=x)

In [ ]:
def upsample_model( shape=1, filters=filters ):
    ''' upsamples SSH '''
    input_img = Input(shape=(None, None, shape))
    upsample = Conv2DTranspose(filters, (4,4), strides=(2,2), padding='same')(input_img)
    return Model(inputs=input_img, outputs=upsample)

In [ ]:
def net():
    simple_conv_1 = simple_conv_model(1)
    simple_conv_2 = simple_conv_model(filters)
    shuffle = shuffle_model()
    #upsample_1 = upsample_1_model()
    upsample_1 = upsample_model(1)
    upsample_2 = upsample_model(filters)
    
    input_hr = Input(shape=(None, None, 1))
    input_lr = Input(shape=(None, None, 1))
    
    # firstPass
    x = simple_conv_1(input_hr)
    y = upsample_1(input_lr)
    hr = Add()([y, x])
    # secondPass
    x = shuffle(x)
    x = simple_conv_2(x)
    y = upsample_2(hr)
    hr = Add()([y, x])
    
    return Model(inputs=[input_lr, input_hr], outputs=hr)

### Train the model

In [ ]:
loss = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
model = net()
model.compile(optimizer=optimizer, loss=loss)